## Data Set Construction

**Functions**

`pd.read_csv`, `pd.read_excel`, `np.diff` or `DataFrame.diff`, `DataFrame.resample`

### Exercise 1

1. Download all available daily data for the S&P 500 and the Hang Seng Index from Yahoo! Finance. 
2. Import both data sets into Python. The final dataset should have a `DateTimeIndex`, and the date
   column should not be part of the `DataFrame`.
3. Construct weekly price series from each, using Tuesday prices (less likely to be a holiday).
4. Construct monthly price series from each using last day in the month.
5. Save the data to the HDF file "equity-indices.h5".


In [1]:
import pandas as pd

sp500 = pd.read_csv("data/GSPC.csv",parse_dates=True,index_col="Date")
hsi = pd.read_csv("data/HSI.csv",parse_dates=True,index_col="Date")

weekly_sp500 = sp500.resample("W-TUE").last()
weekly_hsi = hsi.resample("W-TUE").last()

monthly_sp500 = sp500.resample("M").last()
monthly_hsi = hsi.resample("M").last()

h5file = pd.HDFStore("data/equity-indices.h5", mode="w")
h5file.append("sp500", sp500)
h5file.append("weekly_sp500", weekly_sp500)
h5file.append("monthly_sp500", monthly_sp500)
h5file.append("hsi", sp500)
h5file.append("weekly_hsi", weekly_hsi)
h5file.append("monthly_hsi", monthly_hsi)
h5file.close()


### Exercise 2

Write a function that will correctly aggregate to weekly or monthly respecting the
aggregation rules

* High: `max`
* Low: `min`
* Volume: `sum`

The signature should be:

```python
def yahoo_agg(data, freq):
    <code here>
    return resampled_data
```


In [2]:
def yahoo_agg(data, freq):
    resampler = data.resample(freq)

    high = resampler.High.max()
    low = resampler.Low.min()
    vol = resampler.Volume.sum()
    # Start with last for all columns
    resampled_data = resampler.last()
    # Insert columns that use a different rule
    resampled_data["High"] = high
    resampled_data["Low"] = low
    resampled_data["Volume"] = vol

    return resampled_data


better_monthly_sp500 = yahoo_agg(sp500, "M")

print(monthly_sp500[["High", "Low", "Volume"]].tail())
print(better_monthly_sp500[["High", "Low", "Volume"]].tail())


                   High          Low      Volume
Date                                            
2019-05-31  2768.979980  2750.520020  3981020000
2019-06-30  2943.979980  2929.050049  5420700000
2019-07-31  3017.399902  2958.080078  4623430000
2019-08-31  2940.429932  2913.320068  3008450000
2019-09-30  2999.149902  2982.229980  3186590000
                   High          Low       Volume
Date                                             
2019-05-31  2954.129883  2750.520020  76860120000
2019-06-30  2964.149902  2728.810059  70881390000
2019-07-31  3027.979980  2952.219971  70349470000
2019-08-31  3013.590088  2822.120117  79599440000
2019-09-30  3021.989990  2891.850098  57236800000


### Exercise 3

1. Import the Fama-French benchmark portfolios as well as the 25 sorted portfolios at both the
   monthly and daily horizon from [Ken French"s Data Library](http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html).
   **Note** It is much easier to clean to data file before importing than to find the precise
   command that will load the unmodified data.
2. Import daily FX rate data for USD against AUD, Euro, JPY and GBP from the [Federal Reserve Economic Database (FRED)](http://research.stlouisfed.org/fred2/categories/94). Use Excel rather than csv files.
3. Save the data to the HDF files "fama-french.h5" and "fx.h5"

In [3]:
yen_dollar = pd.read_excel("data/DEXJPUS.xls", index_col="observation_date", skiprows=10)
dollar_aud = pd.read_excel("data/DEXUSAL.xls",index_col="observation_date", skiprows=10)
dollar_euro = pd.read_excel("data/DEXUSEU.xls",index_col="observation_date", skiprows=10)
dollar_pound = pd.read_excel("data/DEXUSUK.xls",index_col="observation_date", skiprows=10)

fx = pd.concat([yen_dollar, dollar_aud, dollar_euro, dollar_pound], axis=1)
print(fx.tail())
fx.to_hdf("data/fx.h5", "fx")

                  DEXJPUS  DEXUSAL  DEXUSEU  DEXUSUK
observation_date                                    
2019-09-16         107.94   0.6865   1.1006   1.2423
2019-09-17         108.12   0.6858   1.1063   1.2493
2019-09-18         108.17   0.6844   1.1061   1.2478
2019-09-19         108.00   0.6795   1.1046   1.2481
2019-09-20         107.95   0.6770   1.1004   1.2489


In [4]:
# These files have all been cleaned to have only the data and headers
ff_5x5 = pd.read_csv("data/25_Portfolios_5x5.CSV", index_col=0)
ff_factors = pd.read_csv("data/F-F_Research_Data_Factors.CSV", index_col=0)
ff = pd.concat([ff_factors, ff_5x5], 1)


dates = []
for value in ff.index:
    # Values are YYYYMM
    dates.append(pd.Timestamp(year=value//100, month=value%100, day=1))
ff.index = dates
print(ff.tail())

# This is a "trick" to get the index to have the last day in the month.
ff = ff.resample("M").last()
print(ff.tail())

ff.to_hdf("data/ff.h5", "ff")

            Mkt-RF   SMB   HML    RF  SMALL LoBM  ME1 BM2  ME1 BM3  ME1 BM4  \
2019-03-01    1.10 -3.13 -4.07  0.19      0.8891  -0.4743  -2.5100  -4.9485   
2019-04-01    3.96 -1.68  1.93  0.21      0.6068   1.6518   2.1840   3.4912   
2019-05-01   -6.94 -1.20 -2.39  0.21     -5.5586  -5.3126  -7.7829  -8.8025   
2019-06-01    6.93  0.33 -1.08  0.18      7.6353   6.2398   4.7677   6.7187   
2019-07-01    1.19 -2.06  0.12  0.19     -3.8504  -1.1600  -0.8926  -1.6496   

            SMALL HiBM  ME2 BM1  ...  ME4 BM1  ME4 BM2  ME4 BM3  ME4 BM4  \
2019-03-01     -3.4235  -0.2319  ...   1.8237   1.0998  -0.9002  -0.5395   
2019-04-01      2.0865   2.8361  ...   3.1410   4.2046   4.3501   5.0762   
2019-05-01     -7.9767  -8.9190  ...  -5.7334  -7.1667  -7.7153  -9.1637   
2019-06-01      4.7008   8.9242  ...   7.2875   9.4043   9.5103   7.5115   
2019-07-01     -2.5149  -3.1247  ...   1.6768   1.0410   1.9332  -0.6625   

            ME4 BM5  BIG LoBM  ME5 BM2  ME5 BM3  ME5 BM4  BIG HiBM  

In [5]:
# These files have all been cleaned to have only the data and headers
ff_5x5_daily = pd.read_csv("data/25_Portfolios_5x5_daily.CSV", index_col=0)
ff_factors_daily = pd.read_csv("data/F-F_Research_Data_Factors_daily.CSV", index_col=0)
ff_daily = pd.concat([ff_factors_daily, ff_5x5_daily], 1)


dates = []
for value in ff_daily.index:
    # Values are YYYYMMDD
    dates.append(pd.Timestamp(year=value//10000, month=(value//100)%100, day=value%100))
ff_daily.index = dates
print(ff_daily.tail())
ff_daily.to_hdf("data/ff_daily.h5", "ff_daily")


            Mkt-RF   SMB   HML     RF  SMALL LoBM  ME1 BM2  ME1 BM3  ME1 BM4  \
2019-07-25   -0.63 -0.69 -0.21  0.009       -1.89    -1.59    -1.02    -1.11   
2019-07-26    0.82  0.39 -0.21  0.009        1.53     1.48     1.29     1.18   
2019-07-29   -0.32 -0.32 -0.47  0.009       -0.34     0.16    -0.89    -0.62   
2019-07-30   -0.17  1.31  0.86  0.009        1.04     1.49     0.84     1.21   
2019-07-31   -1.09  0.13  0.56  0.009       -0.90    -1.76    -0.43    -0.73   

            SMALL HiBM  ME2 BM1  ...  ME4 BM1  ME4 BM2  ME4 BM3  ME4 BM4  \
2019-07-25       -1.59    -1.90  ...    -0.95    -0.72    -0.74    -0.34   
2019-07-26        0.68     1.37  ...     0.84     0.65     0.96     0.17   
2019-07-29       -0.99    -0.82  ...    -0.93    -0.36    -0.24    -0.86   
2019-07-30        2.13     0.86  ...    -0.22    -0.18     0.41     0.95   
2019-07-31       -0.43    -1.44  ...    -0.98    -1.30    -0.68    -0.65   

            ME4 BM5  BIG LoBM  ME5 BM2  ME5 BM3  ME5 BM4  BIG 

### Exercise 3 (Alternative method)

1. Install and use `pandas-datareader` to repeat the previous exercise.

#### Preliminary Step

You must first install the module using 

```
pip install pandas-datareader
``` 

from the command line. Then you can run this code. **Note**: Running this code requires access
to the internet.

In [6]:
import pandas_datareader as pdr

# Conservative start date to get all data
yen_dollar = pdr.get_data_fred("DEXJPUS", start="1950")
dollar_aud = pdr.get_data_fred("DEXUSAL", start="1950")
dollar_euro = pdr.get_data_fred("DEXUSEU", start="1950")
dollar_pound = pdr.get_data_fred("DEXUSUK", start="1950")
fx = pd.concat([yen_dollar, dollar_aud, dollar_euro, dollar_pound], axis=1)
print(fx.tail())
fx.to_hdf("data/fx-pdf.h5", "fx")

            DEXJPUS  DEXUSAL  DEXUSEU  DEXUSUK
DATE                                          
2019-10-14      NaN      NaN      NaN      NaN
2019-10-15   108.83   0.6757   1.1036   1.2737
2019-10-16   108.71   0.6755   1.1075   1.2854
2019-10-17   108.51   0.6826   1.1129   1.2853
2019-10-18   108.44   0.6846   1.1155   1.2904


In [7]:
ff_factors = pdr.get_data_famafrench("F-F_Research_Data_Factors", start="1920")
ff_5x5 = pdr.get_data_famafrench("25_Portfolios_5x5", start="1920")
# The function returns all of the tables in each file.  We want the first, [0]
ff_factors = ff_factors[0]
ff_5x5 = ff_5x5[0]
ff = pd.concat([ff_factors, ff_5x5], 1)
ff.to_hdf("data/ff-pdr.h5", "ff")
print(ff.tail())

         Mkt-RF   SMB   HML    RF  SMALL LoBM  ME1 BM2  ME1 BM3  ME1 BM4  \
Date                                                                       
2019-04    3.96 -1.68  1.93  0.21      0.6068   1.6518   2.1840   3.4912   
2019-05   -6.94 -1.20 -2.39  0.21     -5.5586  -5.3126  -7.7829  -8.8025   
2019-06    6.93  0.33 -1.08  0.18      7.6353   6.2398   4.7677   6.7187   
2019-07    1.19 -2.07  0.14  0.19     -3.8479  -0.8973  -0.8637  -1.6684   
2019-08   -2.58 -2.42 -4.99  0.16     -4.6162  -5.7252  -6.9406  -6.1979   

         SMALL HiBM  ME2 BM1  ...  ME4 BM1  ME4 BM2  ME4 BM3  ME4 BM4  \
Date                          ...                                       
2019-04      2.0865   2.8361  ...   3.1410   4.2046   4.3501   5.0762   
2019-05     -7.9767  -8.9190  ...  -5.7334  -7.1667  -7.7153  -9.1637   
2019-06      4.7008   8.9242  ...   7.2875   9.4043   9.5103   7.5115   
2019-07     -2.6388  -3.1275  ...   1.6579   1.0343   1.9377  -0.6648   
2019-08     -9.2717  -0.5800 